In [1]:
import pandas as pd
import numpy as np
import keras
import scipy
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
from sklearn.model_selection import GroupKFold
from sklearn import linear_model
from lineartree import RandomForestRegressor
from sklearn import metrics
from scipy.signal import savgol_filter
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [2]:
from numpy.random import seed
seed(1)
from tensorflow.keras.utils import set_random_seed
set_random_seed(2)

In [3]:
def rae(actual, predicted):
    actual = np.asarray(actual).reshape(-1,1)
    predicted = np.asarray(predicted).reshape(-1,1)
    numerator = np.sum(np.abs(predicted - actual))
    denominator = np.sum(np.abs(np.mean(actual) - actual))
    return numerator / denominator

In [4]:
def peak_eV_nm(peak_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    return 1239.84193/peak_eV

In [5]:
def FWHM_eV_nm(peak_eV,fwhm_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    fwhm_eV = np.asarray(fwhm_eV).reshape(-1,1)
    return  2*(1239.84193/(peak_eV-fwhm_eV/2) - 1239.84193/(peak_eV))

In [6]:
def predict_dist(X, model, num_samples):
    preds = [model(X, training=True) for _ in range(num_samples)]
    return np.hstack(preds)

In [7]:
def predict_point(X, model, num_samples):
    pred_dist = predict_dist(X, model, num_samples)
    return pred_dist.mean(axis=1)

In [8]:
#Define models

initializer = keras.initializers.HeNormal(seed=1)
opt = keras.optimizers.Adam(learning_rate=0.001)

model_peak = keras.models.Sequential()
model_peak.add(Dense(5, input_shape=(5,), kernel_initializer=initializer, activation='relu'))
model_peak.add(Dense(50,  activation='relu'))
model_peak.add(keras.layers.Dropout(0.2))
model_peak.add(Dense(50,  activation='relu'))
model_peak.add(keras.layers.Dropout(0.2))
model_peak.add(Dense(1, kernel_initializer=initializer))
# Compile model
model_peak.compile(loss='mean_squared_error', optimizer= opt)

model_fwhm = keras.models.Sequential()
model_fwhm.add(Dense(5, input_shape=(5,), kernel_initializer=initializer, activation='relu'))
model_fwhm.add(Dense(50,  activation='relu'))
model_fwhm.add(keras.layers.Dropout(0.2))
model_fwhm.add(Dense(50,  activation='relu'))
model_fwhm.add(keras.layers.Dropout(0.2))
model_fwhm.add(Dense(1, kernel_initializer=initializer))
# Compile model
model_fwhm.compile(loss='mean_squared_error', optimizer= opt)

model_cascade = keras.models.Sequential()
model_cascade.add(Dense(6, input_shape=(6,), kernel_initializer=initializer, activation='relu'))
model_cascade.add(Dense(50,  activation='relu'))
model_cascade.add(keras.layers.Dropout(0.2))
model_cascade.add(Dense(50,  activation='relu'))
model_cascade.add(keras.layers.Dropout(0.2))
model_cascade.add(Dense(1, kernel_initializer=initializer))
# Compile model
model_cascade.compile(loss='mean_squared_error', optimizer= opt)


In [9]:
#Xval on Train/Val set, 10-fold cross validation (Group splits), PL Peak, FWHM (eV), Normalized
# load dataset into Pandas DataFrame (cleaned PL data)
df = pd.read_csv('CdSe PL Train Val.csv')

#normalize input and output data
min_params = pd.Series([0, 0, 3, 200, 0, 400, 15, 1239.84193/620, 0.05], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
max_params = pd.Series([50, 50, 150, 310, 600, 620, 80, 1239.84193/400, 0.4], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] = (df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] - min_params) / (max_params- min_params)

X = df[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s']]
X2 = df[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s','Peak_eV']]
y = df['Peak_eV']
z = df['FWHM_eV']
group = df['Expt. No']

#Split groups for cross validation by Experiment number
gkf = GroupKFold(n_splits=10)
matrix = np.zeros([10,16])
matrix_fwhm = np.zeros([10,16])
matrix_cascade = np.zeros([10,16])

j = 0
#Loop through the 10 splits
for train_index, test_index in gkf.split(X, groups=group):
    #Separate out training and test sets
    X_train = X.iloc[train_index, :]
    X2_train = X2.iloc[train_index, :]
    y_train = y[train_index]
    z_train = z[train_index]
    X_test = X.iloc[test_index, :]
    X2_test = X2.iloc[test_index, :]
    y_test = y[test_index]
    z_test = z[test_index]
    
    #Load weights
    model_peak.load_weights('initialize_weights3.h5')
    model_fwhm.load_weights('initialize_weights3.h5')
    model_cascade.load_weights('initialize_weights4.h5')

    #Train models
    #prediction for Peak
    model_peak.fit(X_train, y_train, epochs=300, validation_data=(X_test, y_test),verbose = 1, use_multiprocessing=False, workers = 100)
    y_pred = model_peak.predict(X_test)
    #prediction for FWHM (direct)
    model_fwhm.fit(X_train, z_train, epochs=300, validation_data=(X_test, z_test),verbose = 1, use_multiprocessing=False, workers = 100)
    y_pred_fwhm = model_fwhm.predict(X_test)
    #prediction for FWHM (cascade)
    model_cascade.fit(X2_train, z_train, epochs=300, validation_data=(X2_test, z_test),verbose = 1, use_multiprocessing=False, workers = 100)
    X_test.insert(5, "Peak_eV", model_peak.predict(X_test), True)
    y_pred_cascade = model_cascade.predict(X_test)

    
    #prediction for Peak with dropout
    X_test = X.iloc[test_index, :]
    X_test_t = K.constant(X_test)
    y_pred_d =  predict_point(X_test_t, model_peak, 1000)
    #prediction for FWHM (direct)
    y_pred_fwhm_d = predict_point(X_test_t, model_fwhm, 1000)
    #prediction for FWHM (cascade)
    X_test.insert(5, "Peak_eV", y_pred_d, True)
    X_test_t2 = K.constant(X_test)
    y_pred_cascade_d = predict_point(X_test_t2, model_cascade, 1000)
    
    #errors in eV (normalized)
    #UN-normalize eV predictions by scaling with ratio

    ratio = (1239.84193/400 - 1239.84193/620)
    ratio2 = 0.35
    matrix[j,0] = metrics.mean_absolute_error(y_test, y_pred)*ratio
    matrix[j,1] = metrics.r2_score(y_test, y_pred)
    matrix[j,2] = rae(y_test, y_pred)
    matrix[j,3] = ratio*np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    matrix[j,4] = metrics.mean_absolute_error(y_test, y_pred_d)*ratio 
    matrix[j,5] = metrics.r2_score(y_test, y_pred_d)
    matrix[j,6] = rae(y_test, y_pred_d)
    matrix[j,7] = ratio*np.sqrt(metrics.mean_squared_error(y_test, y_pred_d))

    matrix_fwhm[j,0] = metrics.mean_absolute_error(z_test, y_pred_fwhm)*ratio2
    matrix_fwhm[j,1] = metrics.r2_score(z_test, y_pred_fwhm)
    matrix_fwhm[j,2] = rae(z_test, y_pred_fwhm)
    matrix_fwhm[j,3] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_fwhm))
    matrix_fwhm[j,4] = metrics.mean_absolute_error(z_test, y_pred_fwhm_d)*ratio2
    matrix_fwhm[j,5] = metrics.r2_score(z_test, y_pred_fwhm_d)
    matrix_fwhm[j,6] = rae(z_test, y_pred_fwhm_d)
    matrix_fwhm[j,7] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_fwhm_d))
    
    matrix_cascade[j,0] = metrics.mean_absolute_error(z_test, y_pred_cascade)*ratio2
    matrix_cascade[j,1] = metrics.r2_score(z_test, y_pred_cascade)
    matrix_cascade[j,2] = rae(z_test, y_pred_cascade)
    matrix_cascade[j,3] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_cascade))
    matrix_cascade[j,4] = metrics.mean_absolute_error(z_test, y_pred_cascade_d)*ratio2
    matrix_cascade[j,5] = metrics.r2_score(z_test, y_pred_cascade_d)
    matrix_cascade[j,6] = rae(z_test, y_pred_cascade_d)
    matrix_cascade[j,7] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_cascade_d))
    
    

    #errors in nm
    #UNnormalize the data first
    y_test = y_test*(1239.84193/400 - 1239.84193/620)+1239.84193/620
    y_pred = y_pred*(1239.84193/400 - 1239.84193/620)+1239.84193/620
    y_pred_d = y_pred_d*(1239.84193/400 - 1239.84193/620)+1239.84193/620
    z_test = z_test*(0.35)+0.05
    y_pred_fwhm = y_pred_fwhm*(0.35)+0.05
    y_pred_fwhm_d = y_pred_fwhm_d*(0.35)+0.05
    y_pred_cascade = y_pred_cascade*(0.35)+0.05
    y_pred_cascade_d = y_pred_cascade_d*(0.35)+0.05
    
    #Calculate data in nm and eV
    matrix[j,8] = metrics.mean_absolute_error(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[j,9] = metrics.r2_score(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[j,10] = rae(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[j,11] = np.sqrt(metrics.mean_squared_error(peak_eV_nm(y_test), peak_eV_nm(y_pred)))
    matrix[j,12] = metrics.mean_absolute_error(peak_eV_nm(y_test), peak_eV_nm(y_pred_d))
    matrix[j,13] = metrics.r2_score(peak_eV_nm(y_test), peak_eV_nm(y_pred_d))
    matrix[j,14] = rae(peak_eV_nm(y_test), peak_eV_nm(y_pred_d))
    matrix[j,15] = np.sqrt(metrics.mean_squared_error(peak_eV_nm(y_test), peak_eV_nm(y_pred_d)))

    matrix_fwhm[j,8] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
    matrix_fwhm[j,9] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
    matrix_fwhm[j,10] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
    matrix_fwhm[j,11] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm)))
    matrix_fwhm[j,12] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d)) 
    matrix_fwhm[j,13] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d))
    matrix_fwhm[j,14] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d)) 
    matrix_fwhm[j,15] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d))) 

    matrix_cascade[j,8] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
    matrix_cascade[j,9] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
    matrix_cascade[j,10] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
    matrix_cascade[j,11] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade)))
    matrix_cascade[j,12] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d))
    matrix_cascade[j,13] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d))
    matrix_cascade[j,14] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d))
    matrix_cascade[j,15] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d)))

    j+=1

matrix_2 = np.zeros([1,32])
matrix_2[0,:16] = matrix.mean(0)[:16]
matrix_2[0,16:32] = np.std(matrix[:,:16], axis=0)

matrix_3 = np.zeros([1,32])
matrix_3[0,:16] = matrix_fwhm.mean(0)[:16]
matrix_3[0,16:32]= np.std(matrix_fwhm[:,:16], axis=0)

matrix_4 = np.zeros([1,32])
matrix_4[0,:16] = matrix_cascade.mean(0)[:16]
matrix_4[0,16:32] = np.std(matrix_cascade[:,:16], axis=0)

np.set_printoptions(precision=5, threshold=5, edgeitems=4, suppress=True)


Epoch 1/300
18/18 [==============================] - 1s 11ms/step - loss: 0.0857 - val_loss: 0.0440
Epoch 2/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0519 - val_loss: 0.0196
Epoch 3/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0338 - val_loss: 0.0173
Epoch 4/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0247 - val_loss: 0.0131
Epoch 5/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0204 - val_loss: 0.0063
Epoch 6/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0150 - val_loss: 0.0056
Epoch 7/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0157 - val_loss: 0.0062
Epoch 8/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0130 - val_loss: 0.0051
Epoch 9/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0123 - val_loss: 0.0055
Epoch 10/300
18/18 [==============================] - 0s 3ms/step - loss: 0.0131 - val_loss: 0.0040
Epoch 11

18/18 [==============================] - 0s 2ms/step - loss: 0.0036 - val_loss: 8.6829e-04
Epoch 83/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0032 - val_loss: 7.6384e-04
Epoch 84/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0037 - val_loss: 8.0778e-04
Epoch 85/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0033 - val_loss: 8.1792e-04
Epoch 86/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0031 - val_loss: 8.7562e-04
Epoch 87/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0031 - val_loss: 9.9800e-04
Epoch 88/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0032 - val_loss: 8.5302e-04
Epoch 89/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0034 - val_loss: 7.7219e-04
Epoch 90/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0031 - val_loss: 9.2644e-04
Epoch 91/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0

18/18 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 0.0011
Epoch 162/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 0.0012
Epoch 163/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0016 - val_loss: 0.0012
Epoch 164/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0017 - val_loss: 9.3576e-04
Epoch 165/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 0.0010
Epoch 166/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 9.2846e-04
Epoch 167/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 0.0011
Epoch 168/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 9.8219e-04
Epoch 169/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 0.0011
Epoch 170/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss:

Epoch 240/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 8.2061e-04
Epoch 241/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 9.2972e-04
Epoch 242/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 243/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 244/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 8.1582e-04
Epoch 245/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 246/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 9.8976e-04
Epoch 247/300
18/18 [==============================] - 0s 2ms/step - loss: 9.8523e-04 - val_loss: 8.6346e-04
Epoch 248/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 8.6946e-04
Epoch 249/300
18/18 [==============================] - 0s 2ms/s

18/18 [==============================] - 0s 2ms/step - loss: 0.0033 - val_loss: 0.0022
Epoch 19/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0031 - val_loss: 0.0037
Epoch 20/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0029 - val_loss: 0.0031
Epoch 21/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0032 - val_loss: 0.0038
Epoch 22/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0031 - val_loss: 0.0030
Epoch 23/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0032 - val_loss: 0.0034
Epoch 24/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0031 - val_loss: 0.0027
Epoch 25/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0030 - val_loss: 0.0032
Epoch 26/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0028 - val_loss: 0.0021
Epoch 27/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0029 - val_loss: 0.0033
Epoch 28/300


...

18/18 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 0.0015
Epoch 223/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 0.0015
Epoch 224/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 0.0015
Epoch 225/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 0.0015
Epoch 226/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 0.0015
Epoch 227/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 0.0015
Epoch 228/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 0.0015
Epoch 229/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0015
Epoch 230/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 0.0015
Epoch 231/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 0.0015
Epoc

Epoch 3/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0065 - val_loss: 0.0024
Epoch 4/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0051 - val_loss: 0.0022
Epoch 5/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0046 - val_loss: 0.0018
Epoch 6/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0044 - val_loss: 0.0020
Epoch 7/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0039 - val_loss: 0.0022
Epoch 8/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0037 - val_loss: 0.0017
Epoch 9/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0036 - val_loss: 0.0018
Epoch 10/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0033 - val_loss: 0.0020
Epoch 11/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0033 - val_loss: 0.0022
Epoch 12/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0033 - val_loss: 0.0017
Epoch 1

Epoch 85/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0016 - val_loss: 0.0016
Epoch 86/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0017 - val_loss: 0.0014
Epoch 87/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0017 - val_loss: 0.0014
Epoch 88/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 0.0015
Epoch 89/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0017 - val_loss: 0.0014
Epoch 90/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0016 - val_loss: 0.0018
Epoch 91/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 0.0014
Epoch 92/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 0.0015
Epoch 93/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0017 - val_loss: 0.0015
Epoch 94/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0016 - val_loss: 0.0016


18/18 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 0.0013
Epoch 167/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0016
Epoch 168/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 0.0013
Epoch 169/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0013
Epoch 170/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0012
Epoch 171/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 0.0013
Epoch 172/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0012
Epoch 173/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0015
Epoch 174/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 0.0014
Epoch 175/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 0.0014
Epoc

18/18 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 0.0014
Epoch 248/300
18/18 [==============================] - 0s 2ms/step - loss: 9.5247e-04 - val_loss: 0.0013
Epoch 249/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0010 - val_loss: 0.0012
Epoch 250/300
18/18 [==============================] - 0s 2ms/step - loss: 9.8764e-04 - val_loss: 0.0013
Epoch 251/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 252/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 0.0016
Epoch 253/300
18/18 [==============================] - 0s 2ms/step - loss: 9.9901e-04 - val_loss: 0.0013
Epoch 254/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 0.0013
Epoch 255/300
18/18 [==============================] - 0s 2ms/step - loss: 0.0010 - val_loss: 0.0012
Epoch 256/300
18/18 [==============================] - 0s 2ms/step - loss: 9.5146e-04 - val_l

In [10]:
print('\033[1m' + 'Model without Dropout at Inference' + '\033[0m')
print('\033[1m' + 'Absorbance Peak' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_2[0,0]), "{:.4f}".format(matrix_2[0,16]))
print("R2 (eV):\t", "{:.4f}".format(matrix_2[0,1]), "{:.4f}".format(matrix_2[0,17]))
print("RAE (eV):\t", "{:.4f}".format(matrix_2[0,2]), "{:.4f}".format(matrix_2[0,18]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_2[0,3]), "{:.4f}".format(matrix_2[0,19]))

print("MAE (nm):\t","{:.4f}".format(matrix_2[0,8]), "{:.4f}".format(matrix_2[0,24]))
print("R2 (nm):\t", "{:.4f}".format(matrix_2[0,9]), "{:.4f}".format(matrix_2[0,25]))
print("RAE (nm):\t","{:.4f}".format(matrix_2[0,10]), "{:.4f}".format(matrix_2[0,26]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_2[0,11]), "{:.4f}".format(matrix_2[0,27]))

print('\033[1m' + 'Absorbance FWHM (direct)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_3[0,0]), "{:.4f}".format(matrix_3[0,16]))
print("R2 (eV):\t", "{:.4f}".format(matrix_3[0,1]), "{:.4f}".format(matrix_3[0,17]))
print("RAE (eV):\t", "{:.4f}".format(matrix_3[0,2]), "{:.4f}".format(matrix_3[0,18]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_3[0,3]), "{:.4f}".format(matrix_3[0,19]))

print("MAE (nm):\t","{:.4f}".format(matrix_3[0,8]), "{:.4f}".format(matrix_3[0,24]))
print("R2 (nm):\t", "{:.4f}".format(matrix_3[0,9]), "{:.4f}".format(matrix_3[0,25]))
print("RAE (nm):\t","{:.4f}".format(matrix_3[0,10]), "{:.4f}".format(matrix_3[0,26]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_3[0,11]), "{:.4f}".format(matrix_3[0,27]))

print('\033[1m' + 'Absorbance FWHM (cascade)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_4[0,0]), "{:.4f}".format(matrix_4[0,16]))
print("R2 (eV):\t", "{:.4f}".format(matrix_4[0,1]), "{:.4f}".format(matrix_4[0,17]))
print("RAE (eV):\t", "{:.4f}".format(matrix_4[0,2]), "{:.4f}".format(matrix_4[0,18]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_4[0,3]), "{:.4f}".format(matrix_4[0,19]))

print("MAE (nm):\t","{:.4f}".format(matrix_4[0,8]), "{:.4f}".format(matrix_4[0,24]))
print("R2 (nm):\t", "{:.4f}".format(matrix_4[0,9]), "{:.4f}".format(matrix_4[0,25]))
print("RAE (nm):\t","{:.4f}".format(matrix_4[0,10]), "{:.4f}".format(matrix_4[0,26]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_4[0,11]), "{:.4f}".format(matrix_4[0,27]))

Model without Dropout at Inference
Absorbance Peak
MAE (eV): 	 0.0561 0.0192
R2 (eV):	 0.7467 0.1492
RAE (eV):	 0.4882 0.1734
RMSE (eV):	 0.0676 0.0221
MAE (nm):	 12.4205 3.9974
R2 (nm):	 0.7418 0.1676
RAE (nm):	 0.4885 0.1830
RMSE (nm):	 14.9197 4.5675
Absorbance FWHM (direct)
MAE (eV): 	 0.0118 0.0038
R2 (eV):	 0.1850 0.2073
RAE (eV):	 0.9011 0.1795
RMSE (eV):	 0.0160 0.0056
MAE (nm):	 3.3689 1.3212
R2 (nm):	 -0.1765 0.6316
RAE (nm):	 1.0330 0.2778
RMSE (nm):	 4.2816 1.7232
Absorbance FWHM (cascade)
MAE (eV): 	 0.0114 0.0041
R2 (eV):	 0.2134 0.2854
RAE (eV):	 0.8540 0.1788
RMSE (eV):	 0.0159 0.0065
MAE (nm):	 3.2903 1.2595
R2 (nm):	 -0.1558 0.6096
RAE (nm):	 0.9960 0.2444
RMSE (nm):	 4.3078 1.8236


In [11]:
print('\033[1m' + 'Model with Dropout at Inference' + '\033[0m')
print('\033[1m' + 'Absorbance Peak' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_2[0,4]), "{:.4f}".format(matrix_2[0,20]))
print("R2 (eV):\t", "{:.4f}".format(matrix_2[0,5]), "{:.4f}".format(matrix_2[0,21]))
print("RAE (eV):\t", "{:.4f}".format(matrix_2[0,6]), "{:.4f}".format(matrix_2[0,22]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_2[0,7]), "{:.4f}".format(matrix_2[0,23]))

print("MAE (nm):\t","{:.4f}".format(matrix_2[0,12]), "{:.4f}".format(matrix_2[0,28]))
print("R2 (nm):\t", "{:.4f}".format(matrix_2[0,13]), "{:.4f}".format(matrix_2[0,29]))
print("RAE (nm):\t","{:.4f}".format(matrix_2[0,14]), "{:.4f}".format(matrix_2[0,30]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_2[0,15]), "{:.4f}".format(matrix_2[0,31]))

print('\033[1m' + 'Absorbance FWHM (direct)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_3[0,4]), "{:.4f}".format(matrix_3[0,20]))
print("R2 (eV):\t", "{:.4f}".format(matrix_3[0,5]), "{:.4f}".format(matrix_3[0,21]))
print("RAE (eV):\t", "{:.4f}".format(matrix_3[0,6]), "{:.4f}".format(matrix_3[0,22]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_3[0,7]), "{:.4f}".format(matrix_3[0,23]))

print("MAE (nm):\t","{:.4f}".format(matrix_3[0,12]), "{:.4f}".format(matrix_3[0,28]))
print("R2 (nm):\t", "{:.4f}".format(matrix_3[0,13]), "{:.4f}".format(matrix_3[0,29]))
print("RAE (nm):\t","{:.4f}".format(matrix_3[0,14]), "{:.4f}".format(matrix_3[0,30]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_3[0,15]), "{:.4f}".format(matrix_3[0,31]))
print('\033[1m' + 'Absorbance FWHM (cascade)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_4[0,4]), "{:.4f}".format(matrix_4[0,20]))
print("R2 (eV):\t", "{:.4f}".format(matrix_4[0,5]), "{:.4f}".format(matrix_4[0,21]))
print("RAE (eV):\t", "{:.4f}".format(matrix_4[0,6]), "{:.4f}".format(matrix_4[0,22]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_4[0,7]), "{:.4f}".format(matrix_4[0,23]))

print("MAE (nm):\t","{:.4f}".format(matrix_4[0,12]), "{:.4f}".format(matrix_4[0,28]))
print("R2 (nm):\t", "{:.4f}".format(matrix_4[0,13]), "{:.4f}".format(matrix_4[0,29]))
print("RAE (nm):\t","{:.4f}".format(matrix_4[0,14]), "{:.4f}".format(matrix_4[0,30]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_4[0,15]), "{:.4f}".format(matrix_4[0,31]))

Model with Dropout at Inference
Absorbance Peak
MAE (eV): 	 0.0552 0.0192
R2 (eV):	 0.7523 0.1472
RAE (eV):	 0.4793 0.1723
RMSE (eV):	 0.0668 0.0221
MAE (nm):	 12.1965 4.0129
R2 (nm):	 0.7485 0.1644
RAE (nm):	 0.4789 0.1818
RMSE (nm):	 14.7309 4.5496
Absorbance FWHM (direct)
MAE (eV): 	 0.0117 0.0038
R2 (eV):	 0.1914 0.2221
RAE (eV):	 0.8941 0.1885
RMSE (eV):	 0.0159 0.0056
MAE (nm):	 3.3381 1.3445
R2 (nm):	 -0.1634 0.6300
RAE (nm):	 1.0207 0.2765
RMSE (nm):	 4.2601 1.7252
Absorbance FWHM (cascade)
MAE (eV): 	 0.0115 0.0040
R2 (eV):	 0.2141 0.2720
RAE (eV):	 0.8645 0.1733
RMSE (eV):	 0.0159 0.0063
MAE (nm):	 3.3027 1.2004
R2 (nm):	 -0.1557 0.5926
RAE (nm):	 1.0033 0.2319
RMSE (nm):	 4.3051 1.7655


In [12]:
np.savetxt("NN+MMCN_PL_Val_peak.csv", matrix, delimiter=",")
np.savetxt("NN+MMCN_PL_Val_fwhm.csv", matrix_fwhm, delimiter=",")
np.savetxt("NN+MMCN_PL_Val_cascade.csv", matrix_cascade, delimiter=",")